This code works through the GEDI subsetter version 0.6.0. The first part of the code works through putting in a job request for multiple AOIs at once. The second part works through a job request for a single AOI

In [22]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')


In [23]:
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd

In [24]:
#Load in Tiled AOI
tiles = gpd.read_file("https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/WestAfricaGrid.geojson")

print(tiles.keys())

Index(['id', 'random', 'geometry'], dtype='object')


Use "random" column for ID to generate features to tile over

In [25]:
list_aois = tiles[tiles.columns[1]].values.tolist()
print(list_aois)

[0.6855097362903453, 0.4522512836485828, 0.23131388830190724, 0.5979918962060293, 0.9954619410018374, 0.7363462239298356, 0.16955097115328577, 0.17510753017660985, 0.3925692959970333, 0.5752152674405157, 0.7412455810081872, 0.7392697493154132, 0.4653903570931611, 0.8354018764581234, 0.38600958865659807, 0.23490126284236712, 0.6936479469170777, 0.895062222650941, 0.789772170178162, 0.5160985973254504, 0.9244779703796152, 0.5150738620210101, 0.894109796377685, 0.06989256159489654, 0.6310652493506097, 0.8557841478391265, 0.3657585583436155, 0.7485440110765288, 0.5672959857440157, 0.33153070200335955, 0.39138004097717916, 0.5822984975127008, 0.4411987903729985, 0.5549448029680308, 0.1657510400319916, 0.658522680224497, 0.5666193031784508, 0.4415834231141531, 0.9917320562143129, 0.5326880936290095, 0.9621639521726443, 0.07476782962538708, 0.7317576801263533, 0.8968646730286827, 0.14280037807636703, 0.582712155141294, 0.03319394615667215, 0.6036670510954576, 0.023375478336133515, 0.175617543

In [26]:
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))


For several AOI's run this code:

In [28]:
#Set up job results to print output during run

import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [ ]:
#Set up input dictionary and import backoff to display run results
#################################

for aoi in list_aois: 
    inputs = dict(
       aoi={"random": aoi},
       doi="L2B",
       lat="geolocation/lat_lowestmode",
       lon="geolocation/lon_lowestmode",
       beams="all",
       columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
       query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
       limit = 1000,
       temporal="-",
       output="-"
    )
    result = maap.submitJob(
        identifier="gedi-subset",
        algo_id="gedi-subset_ubuntu",
        version="0.6.0",
        queue="maap-dps-worker-32gb",
        username="abarenblitt",
    **inputs
    )
    
    job_id = result["job_id"]
    print(job_id)
    
try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

c36435b0-78cb-4e7a-9df6-3185749a0d00
180ae489-42ab-4d1f-807f-fdfebf59ecf9
b5249551-129f-42f8-9284-39cb087152a8
2ea0e402-08ff-4935-bc4c-60f59aeb01f3
bc75ec66-e6b7-41ed-9e88-3d2fbfb99be4
72f208a6-747e-44bf-8d48-e97d88eb62cc
e68151ed-f257-4ebc-9cf5-008b10b095ca
fe1b1fb7-349a-4936-a677-20542ad89b2f
3b175def-99d0-4ee5-aaa0-6a729a958152
19cf7168-90d0-4527-b4b1-6bc37dbc55e6
761e974c-ff14-44c2-9457-4dc6bdd84521
32a70855-285d-4d0d-ad9b-dafdf3e92184
eb285219-df17-41da-9075-2972b2905eb4
0283704f-0143-4e96-85e6-d0bc148b45b7
7b67a029-0707-4408-8c52-9acde655023b
eb94dcf6-4b0b-4ccd-b8ad-11600c6ea67d
7aff0097-33ef-4548-84c4-9fa89739d661
978e837f-4510-43fb-83aa-ce69745b9aff
9bf83250-5a62-4220-b029-5b21301e1b91
5590a353-987c-4da5-9702-ef573deeea56
7e1b63de-6d7c-4c16-bcfb-15b2b93ff79d
0341b22c-cb26-4924-b614-6d1882e5a33d
f08d2887-8ba4-4ce3-bc18-c1e97b904b03
057693c8-cc16-4623-8e44-cc1a0d7ced24
33133b02-2503-4c70-b40f-377791e367fe
919c5e3d-c239-4f1d-bdbe-2da160879b29
7a6c8653-bcf9-4c7b-a7e4-09c6e6fa0bf8
a

INFO:backoff:Backing off wait_for_job(...) for 98.7s (Accepted)


dc814213-acfd-42d5-a489-ca19ca4a567e
a23ef895-d956-4473-a2f9-4ea0772f9dad


In [57]:
#Get job result full text to bug Alex with
 
maap.getJobResult(job_id).text

'<wps:Result xmlns:ows="http://www.opengis.net/ows/2.0" xmlns:schemaLocation="http://schemas.opengis.net/wps/2.0/wps.xsd" xmlns:wps="http://www.opengis.net/wps/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><wps:JobID>c0535b6c-bf0a-4879-baba-c00a255e1438</wps:JobID><wps:Output id="traceback"><wps:Data>+ /app/gedi-subsetter/subset.sh L2B - geolocation/lat_lowestmode geolocation/lon_lowestmode all \'rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,cover_z0,pai_z0,cover_z1,pai_z1,cover_z2,pai_z2,cover_z3,pai_z3,cover_z4,pai_z4,cover_z5,pai_z5,cover_z6,pai_z6,cover_z7,pai_z7,cover_z8,pai_z8,cover_z9,pai_z9,cover_z10,pai_z10,cover_z11,pai_z11,cover_z12,pai_z12,cover_z13,pai_z13,cover_z14,pai_z14,cover_z15,pai_z15,cover_z16,pai_z16,cover_z17,pai_z17,cover_z18,pai_z18,cover_z19,pai_z19,cover_z20,pai_z20,cover_z21,pai_z21,cover_z22,pai_z22,cover_z23,pai_z23,cover_z24,pai_z24,cover_z25,pai_z25,cover_z26,pai_z26,cover_z27,pai_z27,cover_z28,pai_z28,cover_z29,pai_z29\' \'